# Stochastic differential equation model using StochasticDiffEq.jl
Simon Frost (@sdwfrost), 2020-04-27

## Introduction

A stochastic differential equation version of the SIR model is:

- Stochastic
- Continuous in time
- Continuous in state

This implementation uses `StochasticDiffEq.jl`, which has a variety of SDE solvers.

## Libraries

In [ ]:
using DifferentialEquations
using StochasticDiffEq
using DiffEqCallbacks
using Random
using SparseArrays
using DataFrames
using StatsPlots
using BenchmarkTools

## Transitions

We begin by specifying the ODE kernel.

In [ ]:
function sir_ode!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end;

In [ ]:
# Define a sparse matrix by making a dense matrix and setting some values as not zero
A = zeros(3,2)
A[1,1] = 1
A[2,1] = 1
A[2,2] = 1
A[3,2] = 1
A = SparseArrays.sparse(A);

In [ ]:
# Make `g` write the sparse matrix values
function sir_noise!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    ifrac = β*c*I/N*S
    rfrac = γ*I
    du[1,1] = -sqrt(ifrac)
    du[2,1] = sqrt(ifrac)
    du[2,2] = -sqrt(rfrac)
    du[3,2] = sqrt(rfrac)
end;

## Callbacks

It is possible for the stochastic jumps to result in negative numbers of infected individuals, which will throw an error. A `ContinuousCallback` is added that resets infected individuals, `I`, to zero if `I` becomes negative.

In [ ]:
function condition(u,t,integrator) # Event when event_f(u,t) == 0
  u[2]
end;

In [ ]:
function affect!(integrator)
  integrator.u[2] = 0.0
end;

In [ ]:
cb = ContinuousCallback(condition,affect!);

## Time domain

In [ ]:
δt = 0.1
tmax = 40.0
tspan = (0.0,tmax)
t = 0.0:δt:tmax;

## Initial conditions

In [ ]:
u0 = [990.0,10.0,0.0]; # S,I,R

## Parameter values

In [ ]:
p = [0.05,10.0,0.25]; # β,c,γ

## Random number seed

In [ ]:
Random.seed!(1234);

## Running the model

In [ ]:
prob_sde = SDEProblem(sir_ode!,sir_noise!,u0,tspan,p,noise_rate_prototype=A);

The noise process used here is fairly general (non-diagonal and dependent on the states of the system), so the `LambaEM` solver is used.

In [ ]:
sol_sde = solve(prob_sde,LambaEM(),callback=cb);

## Post-processing

We can convert the output to a dataframe for convenience.

In [ ]:
df_sde = DataFrame(sol_sde(t)')
df_sde[!,:t] = t;

## Plotting

We can now plot the results.

In [ ]:
@df df_sde plot(:t,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark solve(prob_sde,LambaEM(),callback=cb)

dix()
```